In [108]:
import cv2, time
import os
import numpy as np
import torch
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

In [109]:
def make_transform():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
                    [0.485, 0.456, 0.406], 
                    [0.229, 0.224, 0.225])

    ])
    
    return transform

def emotion(num):
    emotions = ['neutral','anger','surprise','smile','sad']
    return emotions[num]

def get_face_emotion(file):
    img = cv2.imread(file,cv2.IMREAD_COLOR)

    result_img = img.copy()
    h, w, _ = result_img.shape
    print(f'{w}  {h}')
    blob = cv2.dnn.blobFromImage(result_img, 1.0, (300, 300), [104, 117, 123], False, False)
    net.setInput(blob)

      # inference, find faces
    detections = net.forward()
    num = 0
    # postprocessing
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * w)
            y1 = int(detections[0, 0, i, 4] * h)
            x2 = int(detections[0, 0, i, 5] * w)
            y2 = int(detections[0, 0, i, 6] * h)
            print(f'{x1} {y1} {x2} {y2}')
            #cv2.imwrite(f'../data/new/{num}.jpg',result_img[y1:y2,x1:x2])
            crop = result_img[y1:y2,x1:x2]

            crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
            if crop is None:
                print('pass')
                continue
            
            crop =  Image.fromarray(crop.astype('uint8'), 'RGB')
            crop = transform(crop).unsqueeze(0)
            with torch.no_grad():
                crop = crop.to(device)

                out = model(crop)


                pred = torch.argmax(out,dim=-1)
                pred = pred.type(torch.IntTensor).detach().cpu()

            label = emotion(pred)

          # draw rects
            cv2.rectangle(result_img, (x1, y1), (x2, y2), (255, 0, 0), int(round(h/150)), cv2.LINE_AA)
            cv2.putText(result_img, label, (x1, y1+10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (36,255,12), 2)
    return result_img

In [110]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=5)
state = torch.load('weight_200.pt')
model.load_state_dict(state)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [117]:
model_path = 'models/opencv_face_detector_uint8.pb'
config_path = 'models/opencv_face_detector.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path, config_path)
conf_threshold = 0.7
transfrom = make_transform()
save_path = 'test_result/'
test_path = '../data/assignment/test/img'

result_img = get_face_emotion(f'../data/a_test.jpg')
cv2.imwrite(save_path+'a_test.jpg',result_img)

907  534
72 44 230 231
355 30 522 259
372 314 531 520
69 307 221 525
664 32 826 260
672 304 851 532


True

In [118]:
cv2.imwrite(save_path+'a_test.jpg',result_img)

True